# Data Collection: มาเก็บรวบรวมข้อมูลจากแหล่งต่าง ๆ (DB & REST API) 


In [1]:
! pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 860 kB/s 


In [5]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
import pymysql
import pandas as pd
import requests
from dotenv import load_dotenv

In [7]:
class Config:
  MYSQL_HOST = os.getenv("MYSQL_HOST")
  MYSQL_PORT = int(os.getenv("MYSQL_PORT"))
  MYSQL_USER = os.getenv("MYSQL_USER")
  MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
  MYSQL_DB = os.getenv("MYSQL_DB")
  MYSQL_CHARSET = os.getenv("MYSQL_CHARSET")



## Connect to DB
หลังจากที่มี Credential ของ database แล้วก็สร้าง connection โดยการ connect ไปที่ DB ด้วย Config ของเรา

In [21]:
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

#query ข้อมูลจาก table audible_data

In [11]:
with connection.cursor() as cursor:
  cursor.execute("select * from audible_data")
  result = cursor.fetchall()

result

[{'Book_ID': 1,
  'Book Title': 'Bamboozled by Jesus',
  'Book Subtitle': 'How God Tricked Me into the Life of My Dreams',
  'Book Author': 'Yvonne Orji',
  'Book Narrator': 'Yvonne Orji',
  'Audio Runtime': '6 hrs and 31 mins',
  'Audiobook_Type': 'Unabridged Audiobook',
  'Categories': 'Biographies & Memoirs',
  'Rating': '5',
  'Total No. of Ratings': 47,
  'Price': '$29.65 '},
 {'Book_ID': 2,
  'Book Title': 'Sixth Realm Part 1',
  'Book Subtitle': 'A LitRPG Fantasy Series (The Ten Realms, Book 6)',
  'Book Author': 'Michael Chatfield',
  'Book Narrator': 'Neil Hellegers',
  'Audio Runtime': '13 hrs and 33 mins',
  'Audiobook_Type': 'Unabridged Audiobook',
  'Categories': 'Science Fiction & Fantasy',
  'Rating': '4.5',
  'Total No. of Ratings': 98,
  'Price': '$24.95 '},
 {'Book_ID': 3,
  'Book Title': 'Go Tell the Bees That I Am Gone',
  'Book Subtitle': 'Outlander, Book 9',
  'Book Author': 'Diana Gabaldon',
  'Book Narrator': 'Davina Porter',
  'Audio Runtime': '27 hrs and 30 mi

#result เป็น list ใช้งานลำบา convert to Pandas

In [12]:
audible_data = pd.DataFrame(result)

In [17]:
audible_data.head(3)

,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
1,2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
2,3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99


# อีกวิธีในการอ่าน table ใช้ sql


In [22]:
sql = "SELECT * FROM audible_transaction"
audible_transaction = pd.read_sql(sql, connection)
audible_transaction.head(10)

,timestamp,user_id,book_id,country
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal
1,2021-05-01 00:00:03,561b26c1,829,United States of America
2,2021-05-01 00:00:04,81f149e5,1391,Japan
3,2021-05-01 00:00:07,4f218413,1586,Taiwan
4,2021-05-01 00:00:18,a4066781,300,United States of America
5,2021-05-01 00:00:21,bbbd603a,1377,United States of America
6,2021-05-01 00:00:22,f636754b,342,Canada
7,2021-05-01 00:00:26,0eea4bbd,1600,United States of America
8,2021-05-01 00:00:26,30b900d5,1556,Algeria
9,2021-05-01 00:00:28,a208d42a,790,United States of America


#Join table: audible_transaction & audible_data

In [26]:
transaction = audible_transaction.merge(audible_data, how="left", left_on="book_id", right_on="Book_ID")

#Get data from REST API

In [40]:
url = "https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate"
result_conversion_rate = requests.get(url)
result_conversion_rate = result_conversion_rate.json()
conversion_rate = pd.DataFrame(result_conversion_rate)
#แปลงจาก index เป็น column date ธรรมดาเพื่อความสะดวกในการ join กับ table transaction
conversion_rate = conversion_rate.reset_index().rename(columns={"index": "date"})

#Join the data transaction & conversion rate

In [ ]:
transaction.dtypes

In [ ]:
conversion_rate.dtypes

In [42]:
# ก็อปปี้ column timestamp เก็บเอาไว้ใน column ใหม่ชื่อ date เพื่อที่จะแปลงวันที่เป็น date เพื่อที่จะสามารถนำมา join กับข้อมูลค่าเงินได้
transaction['date'] = transaction['timestamp']

# แปลงให้จาก timestamp เป็น date ในทั้ง 2 dataframe (transaction, conversion_rate)
transaction['date'] = pd.to_datetime(transaction['date']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
final_df = transaction.merge(conversion_rate, how="left", left_on="date", right_on="date")

In [29]:
final_df.head(3)

,timestamp,user_id,book_id,country,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,date,conversion_rate
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,1584,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00,2021-05-01,31.14
1,2021-05-01 00:00:03,561b26c1,829,United States of America,829,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59,2021-05-01,31.14
2,2021-05-01 00:00:04,81f149e5,1391,Japan,1391,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50,2021-05-01,31.14


In [37]:
final_df.dtypes

timestamp               datetime64[ns]
user_id                         object
book_id                          int64
country                         object
Book_ID                          int64
Book Title                      object
Book Subtitle                   object
Book Author                     object
Book Narrator                   object
Audio Runtime                   object
Audiobook_Type                  object
Categories                      object
Rating                          object
Total No. of Ratings           float64
Price                           object
date                            object
conversion_rate                float64
dtype: object

In [31]:
# ลบ $ ออกจาก column price พร้อมแปลงเป็น float
final_df["Price"] = final_df.apply(lambda x: x["Price"].replace("$",""), axis=1)
final_df["Price"] = final_df["Price"].astype(float)

In [32]:
#เพิ่ม column 'THBPrice' ที่เกิดจาก column Price * conversion_rate
final_df["THBPrice"] = final_df["Price"]*final_df["conversion_rate"]

In [33]:
#drop column ที่ไม่จำเป็นต้องใช้ได้ เช่น date ที่ซ้ำซ้อนกับ timestamp
final_df = final_df.drop("date", axis=1)

In [34]:
final_df.head(3)

,timestamp,user_id,book_id,country,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,conversion_rate,THBPrice
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,1584,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,31.14,1089.9000
1,2021-05-01 00:00:03,561b26c1,829,United States of America,829,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,19.59,31.14,610.0326
2,2021-05-01 00:00:04,81f149e5,1391,Japan,1391,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,31.50,31.14,980.9100


In [35]:
#เซฟ final_df เป็นไฟล์ csv 
final_df.to_csv("output.csv",index=False)